In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.openai_functions import OpenAIFunctionsChain
from langchain.prompts import ChatPromptTemplate

In [31]:
prompt = ChatPromptTemplate.from_template("Alex is tall and has blonde hair. Claudia is shorter than Alex and jumps higher than him.")

In [32]:
chain = OpenAIFunctionsChain(
    llm=ChatOpenAI(temperature=0, 
                   model="gpt-3.5-turbo-0613"), 
    prompt=prompt)

In [36]:
inputs = {"entity_schema": 
    {
      "entity_name": "person",
      "properties": {
          "person_name": "string", 
          "height": "string"
              },
        "required": ["person_name", "height"]
    }
         }

In [37]:
# inputs = {"entity_schema": 
#     {
#       "entity_name": "person",
#       "properties": {
#           "person_names": "object", 
#           "heights": "object"
#               },
#         "required": ["person_names", "heights"]
#     }
#          }

In [38]:
chain(inputs)

{'entity_schema': {'entity_name': 'person',
  'properties': {'person_name': 'string', 'height': 'string'},
  'required': ['person_name', 'height']},
 'output': AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_person_info', 'arguments': '{\n  "person_name": "Alex",\n  "height": "tall"\n}'}}, example=False)}

In [28]:
# chain({"__tools__": [
#     {
#       "name": "get_height",
#       "description": "Use this function to save the height of all the people that were mentioned in the passage",
#       "parameters": 
#           {
#               "properties":{
#                 "person_name": {"title": "person_name", "type": "string"},
#                 "height": {"title": "height", "type": "string"},
#               },
#               "required": ["height"],
#               "type": "object",
#           },
#     }
# ]
#       }
#      )